In [ ]:
# More info here: https://github.com/Shivanandroy/simpleT5
# Install simple-T5
!pip install simplet5

In [ ]:
import pandas as pd

# Set local output directory
outputdir = "/content/outputs"

# Load the data - Modified NORNE dataset
train_df = pd.read_csv("https://storage.googleapis.com/notram-public/finetune_datasets/norne_simple/train2.tsv",sep="\t")
test_df = pd.read_csv("https://storage.googleapis.com/notram-public/finetune_datasets/norne_simple/test.tsv",sep="\t")

train_df

In [ ]:
# *********** A FEW USEFUL TRICKS *************

# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text", so this might have to be renames
# df = df.rename(columns={"headlines":"target_text", "text":"source_text"})

# drop not needed  columns
# df = df[['source_text', 'target_text']]

# In T5 you might add a prefix if you want to train it for multiple tasks
# df['source_text'] = "GDPR: " + df['source_text']

# If you need to split the dataset
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(df, test_size=0.2)

# We strongly recommend that you save the models on your Google Drive
# To do that, you will first have to authenticate.
# from google.colab import drive
# drive.mount('/content/drive')
# outputdir = "/content/drive/MyDrive/simpleT5"

In [ ]:
from simplet5 import SimpleT5

# Load a pretrained model
model = SimpleT5()
model.from_pretrained(model_type="mt5", model_name="north/t5_base_NCC")

# Finetune the model
model.train(train_df=train_df[:],
            eval_df=test_df[:], 
            source_max_token_len=64, 
            target_max_token_len=64, 
            batch_size=8, 
            max_epochs=5,
            #early_stopping_patience_epochs = 0,
            outputdir=outputdir,
            use_gpu=True)

In [ ]:
# Load the finetuned model for prediction
# Find the folder where the model is stored, and update the path below
model.load_model("mt5","/content/drive/MyDrive/simpleT5/simplet5-epoch-31-train-loss-0.0057-val-loss-0.0116", use_gpu=True)

text_to_clean="Akkurat nå er Per Egil Kummervold ferdig med sitt foredrag i Oslo."
#text_to_clean="Per i dag er status at Per og Dag har startet firmaet Per og Dag."

model.predict(text_to_clean)